In [1]:
import torch
import torchvision
import cv2
import torchvision.transforms as transforms
import tensorflow as tf
import math
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import heapq
from keras.models import load_model
import glob
import pandas as pd
from tqdm import tqdm

In [2]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

device(type='cpu')

# Corrupted Images

In [ ]:
# The outputs of the models will be saved in resultFolder. 
"resultFolder=\"/data/PaperRelated/14Oct2025/New_multiple_bad_segmentation_masks/\"\n",
    "modelFolder=\"/data/PadChestHeatMapModel/\"\n",
testImageFolder="/data/PaperRelated/14Oct2025/New_multiple_bad_segmentation_masks/"

In [90]:
test_files= glob.glob(testImageFolder + "/*segm.png")
print(len(test_files))

12


In [ ]:
import pandas as pd

files = [
    "    '/data/PHOSP/phosp_measurements.csv',\n",
    "    '/data/PADCHEST/padchest_measurements.csv'\n",
]

dfMain = pd.concat((pd.read_csv(f) for f in files), ignore_index=True)


In [92]:
len(dfMain)

5021

In [93]:
imgPoints=[]
count=0

for imageName in dfMain['image_name']:
    #print(imageName)
    res = any(imageName in imgname for imgname in test_files)
    #print(res)
    
    if res == True :
        count+=1
        #print(imageName)

print("Number of test images: ",len(test_files))
print("Number of test images that are found in csv files: ",count)

Number of test images:  12
Number of test images that are found in csv files:  10


In [ ]:
resultFolder="/data/PaperRelated/14Oct2025/New_multiple_bad_segmentation_masks/"
modelFolder="/data/PadChestHeatMapModel/"
testImageFolder="/data/PaperRelated/14Oct2025/New_multiple_bad_segmentation_masks/"


test_files= glob.glob(testImageFolder + "/*.png")
print(len(test_files))
if not os.path.exists(resultFolder):
    os.makedirs(resultFolder)

In [95]:
len(imgPoints)

10

In [96]:
imgPoints[0]

['040-00010_CR_20210716_152555', 149, 392, 460, 424]

In [97]:
import tensorflow as tf
print(tf.__version__)
print(tf.config.list_physical_devices('GPU'))  # [] means CPU-only


2.20.0
[]


In [ ]:
files = [
    '/data/PHOSP/phosp_measurements.csv',
    '/data/PADCHEST/padchest_measurements.csv'
]

combinedDF = pd.DataFrame()
for file in files:
    tempDF = pd.read_csv(file)
    combinedDF = pd.concat([combinedDF, tempDF], ignore_index=True)

print(len(combinedDF))
combinedDF

In [99]:
class DataGenerator(tf.keras.utils.Sequence):
    
    def __init__(self, imgs, batch_size=1, shuffle=True):
        self.imgs = imgs
        self.batch_size = batch_size
        self.shuffle = shuffle
        self.on_epoch_end()

    def __len__(self):
        return len(self.imgs) // self.batch_size

    def __getitem__(self, index):
        #Get index of images to generate
        indexes = self.indexes[index*self.batch_size:(index+1)*self.batch_size]

        # Generate data
        X = self.__data_generation(indexes)

        return X

    def on_epoch_end(self):
        #Shuffle the data after the generator has run through all samples
        self.indexes = np.arange(len(self.imgs))
        if self.shuffle == True:
            np.random.shuffle(self.indexes)

    def gaussian(self, xL, yL, H, W, sigma=5):
        ##Function that creates the heatmaps##
        channel = [math.exp(-((c - xL) ** 2 + (r - yL) ** 2) / (2 * sigma ** 2)) for r in range(H) for c in range(W)]
        channel = np.array(channel, dtype=np.float32)
        channel = np.reshape(channel, newshape=(H, W))

        return channel

    def __data_generation(self, indexes):
        'Generates data containing batch_size samples'
        X_batch = [self.imgs for i in indexes]
        X_batch = np.array(X_batch)

        return X_batch

In [100]:
def sliding_window(image, stepSize, windowSize):
    windowAreaList=[]
    windowArea=0
    
    for x in range(0, image.shape[0]-windowSize, stepSize):
        for y in range(0, image.shape[1]-windowSize, stepSize):
            windowArea =0
            for num1 in range(0,windowSize):
                for num2 in range(0,windowSize):
                    #print(image[x+num1][y+num2])
                    windowArea+=image[x+num1][y+num2]
                    
            temp=[]
            temp.append(windowArea)
            temp.append(x)
            temp.append(y)
            
            windowAreaList.append(temp)
            
    return windowAreaList

In [101]:
def divideImageGetPoints(img):
    
    # Read the image
    height, width = img.shape

    # Cut the image in half
    width_cutoff = width // 2
    s1 = img[:, :width_cutoff]
    s2 = img[:, width_cutoff:]
    
    wwindowSize=10
    stepSize=5

    windowsLeft = sliding_window(s1, stepSize, wwindowSize)
    windowsRight = sliding_window(s2, stepSize, wwindowSize)
                
    #print("Len Windows Left:", len(windowsLeft))
    #print("Len Windows Right:", len(windowsLeft))
    #print("Len WindowSize:", wwindowSize)

    justWindowAreas=[]
    for i in windowsLeft:
        justWindowAreas.append(i[0])
    largest2nums = heapq.nlargest(1, enumerate(justWindowAreas), key=lambda x: x[1])
    #print(windowsLeft[largest2nums[0][0]])
    point1=windowsLeft[largest2nums[0][0]][2],windowsLeft[largest2nums[0][0]][1]
    cpoint1=int(point1[0]+(wwindowSize/2)),int(point1[1]+(wwindowSize/2))
    
    justWindowAreas=[]
    for i in windowsRight:
        justWindowAreas.append(i[0])
    largest2nums = heapq.nlargest(1, enumerate(justWindowAreas), key=lambda x: x[1])
    #print(windowsRight[largest2nums[0][0]])
    point2=windowsRight[largest2nums[0][0]][2],windowsRight[largest2nums[0][0]][1]
    cpoint2=int(point2[0]+(wwindowSize/2))+256,int(point2[1]+(wwindowSize/2))
    
    #print("Most intensive area centers: ",cpoint1,cpoint2)

    return cpoint1,cpoint2


In [102]:
def PredictModel(imagePath):
    
    img = cv2.imread(imagePath)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
    
    
    stringg=""
    for row in gray:
        for itemIn in range(0,len(row)-1):
            stringg+=str(row[itemIn])+" "
        stringg+=str(row[itemIn+1])
        stringg+="\n"      
        
    img = stringg.split()
    img = ['0' if x == '' else x for x in img]

    image_list = np.array(img,dtype = 'float')
    X_train = image_list.reshape(512,512,1)
    
    #Testing to see if our DataGenerator is working
    X_batch = next(DataGenerator(X_train).__iter__())
    #print(X_batch.shape)

    HeatMapPredictions = heatMapModel.predict(X_batch, verbose=0)
    
    HeatMapresultImage= HeatMapPredictions[1][0].sum(axis=2)
    
    cpoint1, cpoint2 = divideImageGetPoints(HeatMapresultImage)
    
    return cpoint1,cpoint2, HeatMapresultImage, X_batch



In [103]:
resultImgPoints=[]
cnt=0

for imPath in tqdm(test_files):
    #print(imPath)
    
    justName = imPath.split("/")[-1].split("_segm.")[0]
    
    res = any(justName in imgname for imgname in test_files)
    if res == True:
        
        #print(justName)
    
        cpoint1, cpoint2, HeatMapresultImage, X_batch = PredictModel(imPath)

        justName = imPath.split("/")[-1].split(".")[0]
        #print(justName)

        tmp=[]
        tmp.append(justName)
        tmp.append(cpoint1[0])
        tmp.append(cpoint1[1])
        tmp.append(cpoint2[0])
        tmp.append(cpoint2[1])
        resultImgPoints.append(tmp)

        # Turn interactive plotting off
        plt.ioff()

        fig = plt.figure()
        plt.imshow(X_batch.reshape(512,512))
        plt.plot(cpoint1[0], cpoint1[1], "o", markersize=10, color="red")  # og:shorthand for green circle
        plt.plot(cpoint2[0], cpoint2[1], "o", markersize=10, color="red")  # og:shorthand for green circle
        segName= resultFolder + justName + "hm_points.png"
        plt.savefig(segName)
        plt.close(fig)

        cnt+=1



100%|███████████████████████████████████████████| 12/12 [00:07<00:00,  1.70it/s]


In [104]:
print(len(imgPoints))
print(len(resultImgPoints))

10
12


In [105]:
imgPoints[0]

['040-00010_CR_20210716_152555', 149, 392, 460, 424]

In [106]:
resultImgPoints[0]

['040-00010_CR_20210716_152555_segm', 150, 395, 431, 305]

In [107]:
TotalError = 0
icnt=0

row = ["Image_name","Gt_point","Pred_point","left_points_difference","right_points_difference"]

for gtimg in imgPoints:
    for primg in resultImgPoints:
        
        if primg[0].split("_segm")[0] == gtimg[0]:
            
            p1 = [gtimg[1], gtimg[2]]
            q1 = [primg[1], primg[2]]
            err1= math.dist(p1, q1)
            
            p2 = [gtimg[3], gtimg[4]]
            q2 = [primg[3], primg[4]]
            err2= math.dist(p2, q2)
            
            TotalError += err1
            TotalError += err2
            
            icnt+=1
            
            

In [108]:
icnt

10

In [109]:
ErrorRate = (TotalError / (icnt*512))

In [110]:
ErrorRate

0.2507167700027754

# Fixed Images

In [ ]:
# The outputs of the models will be saved in resultFolder. 
resultFolder="/data/badGradCamExample/new_curved_one/"
testImageFolder="/data/badGradCamExample/new_curved_one/"

In [88]:
test_files= glob.glob(testImageFolder + "/*predicted_gray.png")
print(len(test_files))

0


In [ ]:
import pandas as pd

files = [
    '/data/PHOSP/phosp_measurements.csv',
    '/data/PADCHEST/padchest_measurements.csv'
]

dfMain = pd.concat((pd.read_csv(f) for f in files), ignore_index=True)


In [14]:
len(dfMain)

5021

In [15]:
imgPoints=[]
count=0

for imageName in dfMain['image_name']:
    #print(imageName)
    res = any(imageName in imgname for imgname in test_files)
    #print(res)
    
    if res == True :
        count+=1

print("Number of test images: ",len(test_files))
print("Number of test images that are found in csv files: ",count)

Number of test images:  1
Number of test images that are found in csv files:  1


In [16]:
imgPoints=[]
for idx in range(0,len(dfMain)):
    res = any(dfMain['image_name'][idx] in imgname for imgname in test_files)
    if res == True :
        tmp=[]
        tmp.append(dfMain['image_name'][idx])
        tmp.append(int(dfMain['ref_point_fst_lung'][idx].split(', ')[0].split('(')[-1]))
        tmp.append(int(dfMain['ref_point_fst_lung'][idx].split(', ')[1].split(')')[0]))
        tmp.append(int(dfMain['ref_point_snd_lung'][idx].split(', ')[0].split('(')[-1]))
        tmp.append(int(dfMain['ref_point_snd_lung'][idx].split(', ')[1].split(')')[0]))
        imgPoints.append(tmp)

In [17]:
len(imgPoints)

1

In [18]:
imgPoints[0]

['041-00186_DX_20210414_104200', 91, 362, 374, 376]

In [20]:
import tensorflow as tf
from tensorflow.keras.layers import Conv2DTranspose as _Conv2DTranspose

class Conv2DTransposeCompat(_Conv2DTranspose):
    def __init__(self, *args, groups=1, **kwargs):
        super().__init__(*args, **kwargs)

model_path = modelFolder + 'esma-heatmap-weights-improvement-100.hdf5'
heatMapModel = tf.keras.models.load_model(
    model_path,
    custom_objects={'Conv2DTranspose': Conv2DTransposeCompat},
    compile=False
)


In [21]:
class DataGenerator(tf.keras.utils.Sequence):
    
    def __init__(self, imgs, batch_size=1, shuffle=True):
        self.imgs = imgs
        self.batch_size = batch_size
        self.shuffle = shuffle
        self.on_epoch_end()

    def __len__(self):
        return len(self.imgs) // self.batch_size

    def __getitem__(self, index):
        #Get index of images to generate
        indexes = self.indexes[index*self.batch_size:(index+1)*self.batch_size]

        # Generate data
        X = self.__data_generation(indexes)

        return X

    def on_epoch_end(self):
        #Shuffle the data after the generator has run through all samples
        self.indexes = np.arange(len(self.imgs))
        if self.shuffle == True:
            np.random.shuffle(self.indexes)

    def gaussian(self, xL, yL, H, W, sigma=5):
        ##Function that creates the heatmaps##
        channel = [math.exp(-((c - xL) ** 2 + (r - yL) ** 2) / (2 * sigma ** 2)) for r in range(H) for c in range(W)]
        channel = np.array(channel, dtype=np.float32)
        channel = np.reshape(channel, newshape=(H, W))

        return channel

    def __data_generation(self, indexes):
        'Generates data containing batch_size samples'
        X_batch = [self.imgs for i in indexes]
        X_batch = np.array(X_batch)

        return X_batch

In [22]:
def sliding_window(image, stepSize, windowSize):
    windowAreaList=[]
    windowArea=0
    
    for x in range(0, image.shape[0]-windowSize, stepSize):
        for y in range(0, image.shape[1]-windowSize, stepSize):
            windowArea =0
            for num1 in range(0,windowSize):
                for num2 in range(0,windowSize):
                    #print(image[x+num1][y+num2])
                    windowArea+=image[x+num1][y+num2]
                    
            temp=[]
            temp.append(windowArea)
            temp.append(x)
            temp.append(y)
            
            windowAreaList.append(temp)
            
    return windowAreaList

In [23]:
def divideImageGetPoints(img):
    
    # Read the image
    height, width = img.shape

    # Cut the image in half
    width_cutoff = width // 2
    s1 = img[:, :width_cutoff]
    s2 = img[:, width_cutoff:]
    
    wwindowSize=10
    stepSize=5

    windowsLeft = sliding_window(s1, stepSize, wwindowSize)
    windowsRight = sliding_window(s2, stepSize, wwindowSize)
                
    #print("Len Windows Left:", len(windowsLeft))
    #print("Len Windows Right:", len(windowsLeft))
    #print("Len WindowSize:", wwindowSize)

    justWindowAreas=[]
    for i in windowsLeft:
        justWindowAreas.append(i[0])
    largest2nums = heapq.nlargest(1, enumerate(justWindowAreas), key=lambda x: x[1])
    #print(windowsLeft[largest2nums[0][0]])
    point1=windowsLeft[largest2nums[0][0]][2],windowsLeft[largest2nums[0][0]][1]
    cpoint1=int(point1[0]+(wwindowSize/2)),int(point1[1]+(wwindowSize/2))
    
    justWindowAreas=[]
    for i in windowsRight:
        justWindowAreas.append(i[0])
    largest2nums = heapq.nlargest(1, enumerate(justWindowAreas), key=lambda x: x[1])
    #print(windowsRight[largest2nums[0][0]])
    point2=windowsRight[largest2nums[0][0]][2],windowsRight[largest2nums[0][0]][1]
    cpoint2=int(point2[0]+(wwindowSize/2))+256,int(point2[1]+(wwindowSize/2))
    
    #print("Most intensive area centers: ",cpoint1,cpoint2)

    return cpoint1,cpoint2


In [24]:
def PredictModel(imagePath):
    
    img = cv2.imread(imagePath)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
    
    
    stringg=""
    for row in gray:
        for itemIn in range(0,len(row)-1):
            stringg+=str(row[itemIn])+" "
        stringg+=str(row[itemIn+1])
        stringg+="\n"      
        
    img = stringg.split()
    img = ['0' if x == '' else x for x in img]

    image_list = np.array(img,dtype = 'float')
    X_train = image_list.reshape(512,512,1)
    
    #Testing to see if our DataGenerator is working
    X_batch = next(DataGenerator(X_train).__iter__())
    #print(X_batch.shape)

    HeatMapPredictions = heatMapModel.predict(X_batch, verbose=0)
    
    HeatMapresultImage= HeatMapPredictions[1][0].sum(axis=2)
    
    cpoint1, cpoint2 = divideImageGetPoints(HeatMapresultImage)
    
    return cpoint1,cpoint2, HeatMapresultImage, X_batch



In [25]:
resultImgPoints=[]
cnt=0

for imPath in tqdm(test_files):
    #print(imPath)
    
    justName = imPath.split("/")[-1].split("_segm.")[0]
    
    res = any(justName in imgname for imgname in test_files)
    if res == True:
        
        #print(justName)
    
        cpoint1, cpoint2, HeatMapresultImage, X_batch = PredictModel(imPath)

        justName = imPath.split("/")[-1].split(".")[0]
        #print(justName)

        tmp=[]
        tmp.append(justName)
        tmp.append(cpoint1[0])
        tmp.append(cpoint1[1])
        tmp.append(cpoint2[0])
        tmp.append(cpoint2[1])
        resultImgPoints.append(tmp)

        # Turn interactive plotting off
        plt.ioff()

        fig = plt.figure()
        plt.imshow(X_batch.reshape(512,512))
        plt.plot(cpoint1[0], cpoint1[1], "o", markersize=10, color="red")  # og:shorthand for green circle
        plt.plot(cpoint2[0], cpoint2[1], "o", markersize=10, color="red")  # og:shorthand for green circle
        segName= resultFolder + justName + "hm_points.png"
        plt.savefig(segName)
        plt.close(fig)

        cnt+=1



100%|█████████████████████████████████████████████| 1/1 [00:00<00:00,  1.31it/s]


In [26]:
print(len(imgPoints))
print(len(resultImgPoints))

1
1


In [27]:
imgPoints[0]

['041-00186_DX_20210414_104200', 91, 362, 374, 376]

In [28]:
resultImgPoints[0]

['041-00186_DX_20210414_104200_segm_predicted_gray', 120, 350, 376, 370]

In [29]:
TotalError = 0
icnt=0

row = ["Image_name","Gt_point","Pred_point","left_points_difference","right_points_difference"]

for gtimg in imgPoints:
    for primg in resultImgPoints:
        
        if primg[0].split("_segm")[0] == gtimg[0]:
            
            p1 = [gtimg[1], gtimg[2]]
            q1 = [primg[1], primg[2]]
            err1= math.dist(p1, q1)
            
            p2 = [gtimg[3], gtimg[4]]
            q2 = [primg[3], primg[4]]
            err2= math.dist(p2, q2)
            
            TotalError += err1
            TotalError += err2
            
            icnt+=1
            
            

In [30]:
icnt

1

In [31]:
ErrorRate = (TotalError / (icnt*512))

In [32]:
ErrorRate

0.07365090815095154